In [ ]:
## Install packages
!uv pip install pydantic
!uv pip install instructor
!uv pip install pymupdf
!uv pip install pymupdf4llm

In [2]:
import getpass
import os
import pathlib

import instructor
import pymupdf
import pymupdf4llm
from openai import OpenAI
from pydantic import BaseModel

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [3]:
_API_KEY = getpass.getpass("Your API key: \n")

Your API key: 
 ········


In [4]:
client = instructor.from_provider(
    "openai/gpt-4o-mini", mode=instructor.Mode.RESPONSES_TOOLS, api_key=_API_KEY
)

In [5]:
class Response(BaseModel):
    message: str

In [6]:
def get_response(query, url):
    return client.responses.create(
        response_model=Response,
        input=[
            {
                "role": "developer",
                "content": "You are a world-class expert. You will analyze the document at the given URL and answer the query.",
            },
            {"role": "user", "content": f"url: {url}"},
            {
                "role": "user",
                "content": f"{query}",
            },
        ],
    )

In [7]:
r = get_response(
    "Who is the highest paid employee?",
    "https://budgetandfinance.psu.edu/sites/budgetandfinance/files/right_to_know_2023_0.pdf",
)

In [8]:
print(r.message)

I can't access external documents or URLs directly. However, if you can provide the relevant text or data from the document, I'd be happy to help you analyze it!


In [9]:
# Data from https://budgetandfinance.psu.edu/budget-allocations
# Extracted data in data/data-activity-03/

table = """
| UP Colleges                           | FY27 Budget | FY28 Budget | FY28 Change | % Change |
| -------------------------------------- | -----------: | -----------: | -----------: | --------: |
| Agricultural Sciences (General Funds) |  32,688,075 |  33,096,753 |     408,678 |    1.25% |
| Arts & Architecture                   |  43,699,368 |  43,035,574 |    -663,794 |   -1.52% |
| Business, Smeal                       |  96,862,631 |  96,262,029 |    -600,602 |   -0.62% |
| Communications, Bellisario            |  25,618,729 |  26,556,184 |     937,455 |    3.66% |
| Earth & Mineral Sciences              |  53,074,125 |  50,991,807 |  -2,082,318 |   -3.92% |
| Education                             |  36,948,353 |  35,743,069 |  -1,205,284 |   -3.26% |
| Engineering                           | 131,744,915 | 136,627,116 |   4,882,201 |    3.71% |
| Health & Human Development            |  72,754,814 |  72,283,078 |    -471,736 |   -0.65% |
| Information Sciences & Technology     |  37,301,073 |  38,099,534 |     798,461 |    2.14% |
| International Affairs                 |   2,142,627 |   2,032,131 |    -110,496 |   -5.16% |
| Liberal Arts                          | 174,659,327 | 170,210,893 |  -4,448,434 |   -2.55% |
| Nursing, Nese                         |  17,444,578 |  16,829,172 |    -615,406 |   -3.53% |
| Science, Eberly                       | 136,627,671 | 134,716,659 |  -1,911,012 |   -1.40% |
| UP Colleges                           | 861,566,286 | 856,484,001 |  -5,082,285 |   -0.59% |
"""

In [ ]:
class BudgetAnswer(BaseModel):
    amount: float

In [11]:
def extract_from_table(query, table):
    return client.responses.create(
        response_model=BudgetAnswer,
        input=[
            {
                "role": "developer",
                "content": "You are a world-class finance expert. You will analyze the given markdown tables and answer the query.",
            },
            {"role": "user", "content": f"tables: {table}"},
            {
                "role": "user",
                "content": f"query: {query}",
            },
        ],
    )

In [13]:
q = "How much money is allocated for Information Sciences & Technology in FY28?"
r = extract_from_table(q, table)

In [14]:
print(r.amount)

38099534.0


In [15]:
# If you want to pretty print:
print(f"{r.amount:0,.2f}")

38,099,534.00


# Todo: Which UP colleges have the highest positive and negative percentage changes in 2027 and 2028?

# Todo: Analyze the allocation for other units


Steps:

1. Add data from: `Commonwealth Campuses` (see [data/data-activity-03](https://raw.githubusercontent.com/hcai-ist/activity-spring-2026/refs/heads/main/data/data-activity-03/budget-all-tables.md))
2. Come up with 3 questions to answer
3. Complete code to answer these questions
4. Check answers for accuracy

# Using data from pdf

I'd like to use data from [Right To Know Law Reports from Penn State](https://budgetandfinance.psu.edu/public-reports#:~:text=Right%20To%20Know%20Law%20Report).

# What if we try to extract data directly from the pdf file?

Two steps:
* extract textual data from the pdf using an external library `pymupdf4llm`
* use the textual data as part of the prompt

First, we will have to make the pdf file available.

1. Download the [psu-right-to-know-form-990.pdf](https://budgetandfinance.psu.edu/sites/budgetandfinance/files/right_to_know_2024.pdf) file

2. Upload the file in `Jupyter` (using the ⬆ button)



In [25]:
## We will extract page 45–46
# Note that `pymupdf4llm` uses 0-based index for pages
highest_paid = pymupdf4llm.to_markdown("right_to_know_2024.pdf", pages=[44, 45])

In [27]:
q = "How much did James Franklin earn?"
r = extract_from_table(q, highest_paid)

In [28]:
# If you want to pretty print:
print(f"{r.amount:0,.2f}")

8,628,807.00


# ToDos:

* How much did the CEO of Penn State Health earn?
* What was the biggest source of PSU revenue?
* How much did Penn State spend on advertising and promotion?
* How much did IST Dean earn?